In [1]:
%matplotlib inline
import torch
import torchvision
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu' # CPU or GPU

train_dataset = torchvision.datasets.MNIST('./', train=True, download=True) # get train dataset
test_dataset = torchvision.datasets.MNIST('./', train=False) # get test dataset


c:\Users\hodooai\anaconda3\envs\mnist\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_train_orig = train_dataset.train_data  #train image, [Type image, width, height]
y_train_orig = train_dataset.train_labels #train labels, [Type labels]
x_test_orig = test_dataset.test_data 
y_test_orig = test_dataset.test_labels
print('x_train shape : ', x_train_orig.shape) 
print('y_train shape : ', y_train_orig.shape)
print('x_test  shape : ', x_test_orig.shape)
print('y_test  shape : ', y_test_orig.shape)


x_train shape :  torch.Size([60000, 28, 28])
y_train shape :  torch.Size([60000])
x_test  shape :  torch.Size([10000, 28, 28])
y_test  shape :  torch.Size([10000])


c:\Users\hodooai\anaconda3\envs\mnist\lib\site-packages\torchvision\datasets\mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
c:\Users\hodooai\anaconda3\envs\mnist\lib\site-packages\torchvision\datasets\mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
c:\Users\hodooai\anaconda3\envs\mnist\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\hodooai\anaconda3\envs\mnist\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [4]:
#normalization , 픽셀값 255 -> 0과 1사이로 값을 줄이기 위해서  255로 나눔
x_train = (x_train_orig[10000:] / 255.).to(device=device) # 학습자체에 쓰인 데이터, 수학 비교 -> 문제 풀이 존나 학습 // #To device는 GPU로 보냄
y_train = y_train_orig[10000:].to(device=device)
x_val = (x_train_orig[:10000] / 255.).to(device=device) # 학습과정 안에서 평가를 위해 쓰인 데이터 //  수학 비교 -> 존나 학습한거 새 문제 풀어보기 및 정답 확인
y_val = y_train_orig[:10000].to(device=device)
x_test = x_test_orig / 255. # 학습 후 최종 평가
y_test = y_test_orig
 
print('x_train shape : ', x_train.shape)
print('y_train shape : ', y_train.shape)
print('x_train shape : ', x_val.shape)
print('x_type : ', type(x_val))
print('y_train shape : ', y_val.shape)
   
print('x_test  shape : ', x_test.shape)
print('y_test  shape : ', y_test.shape)

x_train shape :  torch.Size([50000, 28, 28])
y_train shape :  torch.Size([50000])
x_train shape :  torch.Size([10000, 28, 28])
y_train shape :  torch.Size([10000])
x_test  shape :  torch.Size([10000, 28, 28])
y_test  shape :  torch.Size([10000])


In [5]:
trainingset = torch.utils.data.TensorDataset(x_train, y_train) #데이터셋
train_loader = torch.utils.data.DataLoader(trainingset, batch_size=32, shuffle=True) # 전체 이미지를 분할하여 불러오는 로더

1번 테스트

In [12]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(784, 256) # 784란 28(width) x 28(height)
        self.fc2 = torch.nn.Linear(256, 256)
        self.fc3 = torch.nn.Linear(256, 256)
        self.fc4 = torch.nn.Linear(256, 128)
        self.fc5 = torch.nn.Linear(128, 128)
        self.fc6 = torch.nn.Linear(128, 10)

    def forward(self, x): # fully connected
        x = x.view(-1, 784) #-1 = 아무거나, 784 = 28 x 28 // 1 784, 2 392 ... 이런너낌스
        x = torch.nn.functional.relu(self.fc1(x)) #relu +신호는 그대로 -신호는 차단
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.relu(self.fc3(x))
        x = torch.nn.functional.relu(self.fc4(x))
        x = torch.nn.functional.relu(self.fc5(x))
        x = torch.nn.functional.dropout(x, training=self.training) #dropout : layer를 랜덤하게 삭제해줌
        x = self.fc6(x) 
        return x
m1 = Net()
m1.cuda() # GPU로 전송


criterion = torch.nn.CrossEntropyLoss() # criterion 목적함수
optimizer = torch.optim.Adam(m1.parameters(), lr=0.001) #Adam이라는 최적화 기법

# 배치 -> 이미지를 나눠 학습
# epoch -> 전체이미지 반복학습
for epoch in range(10):
    start = time.time()
    total_loss = 0

    for xb, yb in train_loader:
        #xb, yb = torch.autograd.Variable(xb), torch.autograd.Variable(yb)

        pred = m1(xb) #pred = prediction
        loss = criterion(pred, yb)

        loss.backward() #바뀔 값들을 연산
        optimizer.step() # 실제 바뀐값을 적용
        optimizer.zero_grad() # ?검색해보셈 

        total_loss += loss.item()

    with torch.no_grad(): #1에폭당 한번 평가 여기선 val데이터를 사용해야함 왜냐면 학습한걸로 평가하면 ㅄ이기때문이다
        pred = m1(x_val)
     
        acc = pred.data.max(1)[1].eq(y_val.data).sum()/len(x_val) * 100
        loss = criterion(pred, y_val)
    print(f"{time.time() -  start} sec - loss : {loss} / acc : {acc}")


5.79616117477417 sec - loss : 0.18525858223438263 / acc : 95.29000091552734
5.564436912536621 sec - loss : 0.14928655326366425 / acc : 96.31999969482422
6.279510259628296 sec - loss : 0.11206672340631485 / acc : 97.18000030517578
6.180745601654053 sec - loss : 0.11277177929878235 / acc : 97.18000030517578
5.5314226150512695 sec - loss : 0.1400911957025528 / acc : 96.93000030517578
5.577991247177124 sec - loss : 0.10267786681652069 / acc : 97.47999572753906
5.78205132484436 sec - loss : 0.11861927807331085 / acc : 97.5999984741211
5.897591829299927 sec - loss : 0.11700544506311417 / acc : 97.39999389648438
6.005919694900513 sec - loss : 0.11018732935190201 / acc : 97.54999542236328
5.7625977993011475 sec - loss : 0.1278049498796463 / acc : 97.77999877929688


In [7]:
pred = m1(x_test.to(device=device))
acc = pred.data.max(1)[1].eq(y_test.to(device=device).data).sum()/len(x_test) * 100
loss = criterion(pred, y_test.to(device=device))
print(f"loss : {loss} / acc : {acc}")


loss : 0.11388404667377472 / acc : 97.85999298095703


In [8]:
class CNN_Model(torch.nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 16, 2, 1, padding=1)
        self.fc1 = torch.nn.Linear(784, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.Flatten()(x)
        x = self.fc1(x)
        return x
    


In [9]:
m2 = CNN_Model()
m2.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(m2.parameters(), lr= 0.001)


In [10]:
for epoch in range(5):
    start = time.time()
    total_loss = 0
    
    for xb, yb in train_loader:        
        pred = m2(xb)
        loss = criterion(pred, yb.long())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
    
    with torch.no_grad():
        pred = m2(x_train)
        acc = pred.data.max(1)[1].eq(y_train.data).sum()/len(x_train) * 100
        loss = criterion(pred, y_train)
    print(f"{time.time() - start} sec - loss : {loss} / acc : {acc}")


5.29483699798584 sec - loss : 0.10558062791824341 / acc : 96.8219985961914
3.813631534576416 sec - loss : 0.06708876043558121 / acc : 97.94599914550781
3.867177963256836 sec - loss : 0.05351058021187782 / acc : 98.3899917602539
3.916050910949707 sec - loss : 0.04785405099391937 / acc : 98.45800018310547
3.9998743534088135 sec - loss : 0.04052738472819328 / acc : 98.74800109863281


In [11]:
pred = m2(x_test.to(device=device))
acc = pred.data.max(1)[1].eq(y_test.to(device=device).data).sum()/len(x_test) * 100
loss = criterion(pred, y_test.to(device=device))
print(f"loss : {loss} / acc : {acc}")

loss : 0.053827498108148575 / acc : 98.28999328613281


2번 테스트